### Language Analysis

In [1]:
from sklearn.metrics import (
    plot_precision_recall_curve, precision_recall_curve, confusion_matrix, classification_report)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [37]:
#languages
language_df = pd.read_csv("Data/version_7/context_all_lbl_v7.csv")
language_df = language_df[["mid", "language", "urgency"]]
language_df.to_csv("Data/version_7/multiclass_labels.csv")
language_df

,mid,language,urgency
0,829,english,3
1,831,english,3
2,834,english,3
3,835,english,3
4,838,sheng;english;swahili,3
...,...,...,...
11124,220273,swahili,5
11125,220327,english,5
11126,220355,luo,5
11127,220423,english,5


In [3]:
Exp = {"Exp23":"Experiment_23/output/exp23_results.csv",
       "Exp24":"Experiment_24/output/exp24_results.csv",
       "Exp25":"Experiment_25/output/exp25_results.csv",
       "Exp26":"Experiment_26/output/exp26_results.csv",
       "Exp27":"Experiment_27/output/exp27_results.csv",
       "Exp28":"Experiment_28/output/exp28_results.csv",
       "Exp29":"Experiment_29/output/exp29_results.csv",
       "Exp30":"Experiment_30/output/exp30_results.csv",
       "Exp31":"Experiment_31/output/exp31_results.csv",
       "Exp34":"Experiment_34/output/exp34_results.csv",
       "Exp35":"Experiment_35/output/exp35_results.csv",
       "Exp38":"Experiment_38/output/exp38_results.csv"
      }
exp_report = pd.DataFrame()
for exp, path in Exp.items():
    
    # Results Analyis
    results_df = pd.read_csv(path)
    results_df = results_df.assign(pred = np.where(results_df.score > 0.5, 1, 0))
    #Add language to results dataset
    results_df = results_df.merge(language_df, how="left", on="mid" )
    results_groups = results_df.groupby("language")
    english_df = results_groups.get_group('english')
    luo_df = results_groups.get_group('luo')
    sheng_df = results_groups.get_group('sheng')
    swahili_df = results_groups.get_group('swahili')

    dfs = {"english":english_df, "swahili":swahili_df, "luo":luo_df, "sheng":sheng_df,}
    class_report = pd.DataFrame()

    for lang, df in dfs.items():
        report = classification_report(df.label, df.pred, output_dict=True)
        df = pd.DataFrame(report).transpose()
        df['language'] = lang

        class_report = pd.concat([class_report, df])

    class_report = class_report.rename_axis('label').reset_index()
    class_report['Experiment'] = exp
    exp_report = pd.concat([exp_report, class_report])

exp_report = exp_report.reset_index()

C:\Users\nngao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\nngao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\nngao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [4]:
#convert support to integer
exp_report['support'] = exp_report['support'].astype(int)

In [24]:
#highest recalls for positive class
recall_high = exp_report.loc[(exp_report['recall'] > 0.45) & (exp_report['label'] == '1'),:].sort_values(by=['recall'])
recall_high.to_csv("Reports/languages/high_recall.csv")
recall_high

,index,label,precision,recall,f1-score,support,language,Experiment
166,6,1,0.548571,0.457143,0.498701,210,swahili,Exp31
136,16,1,0.714286,0.476190,0.571429,21,sheng,Exp29
81,1,1,0.524444,0.493724,0.508621,239,english,Exp27
96,16,1,0.478261,0.523810,0.500000,21,sheng,Exp27
156,16,1,0.500000,0.523810,0.511628,21,sheng,Exp30
236,16,1,0.550000,0.523810,0.536585,21,sheng,Exp38
116,16,1,0.521739,0.571429,0.545455,21,sheng,Exp28
176,16,1,0.666667,0.571429,0.615385,21,sheng,Exp31
196,16,1,0.666667,0.666667,0.666667,21,sheng,Exp34


In [25]:
#highest precisions for positive class
precision_high = exp_report.loc[(exp_report['precision'] > 0.55) & (exp_report['label'] == '1'),:].sort_values(by=['precision'])
precision_high.to_csv("Reports/languages/high_precision.csv")
precision_high

,index,label,precision,recall,f1-score,support,language,Experiment
206,6,1,0.552941,0.447619,0.494737,210,swahili,Exp35
211,11,1,0.555556,0.277778,0.370370,18,luo,Exp35
216,16,1,0.642857,0.428571,0.514286,21,sheng,Exp35
11,11,1,0.666667,0.111111,0.190476,18,luo,Exp23
176,16,1,0.666667,0.571429,0.615385,21,sheng,Exp31
196,16,1,0.666667,0.666667,0.666667,21,sheng,Exp34
136,16,1,0.714286,0.476190,0.571429,21,sheng,Exp29


In [26]:
#mBERT - no context
mbert_none = exp_report[(exp_report.Experiment.isin(["Exp23"])) & ~exp_report.label.isin(["accuracy","macro avg","weighted avg"])]
mbert_none.to_csv("Reports/languages/mbert_none.csv")
mbert_none

,index,label,precision,recall,f1-score,support,language,Experiment
0,0,0,0.851777,0.880378,0.865841,953,english,Exp23
1,1,1,0.449275,0.389121,0.417040,239,english,Exp23
5,5,0,0.789928,0.892683,0.838168,615,swahili,Exp23
6,6,1,0.492308,0.304762,0.376471,210,swahili,Exp23
10,10,0,0.854545,0.989474,0.917073,95,luo,Exp23
11,11,1,0.666667,0.111111,0.190476,18,luo,Exp23
15,15,0,0.867925,0.938776,0.901961,98,sheng,Exp23
16,16,1,0.538462,0.333333,0.411765,21,sheng,Exp23


In [27]:
#mBERT Nurse context languages (pretrained) - labeled
mbert_nurse = exp_report[(exp_report.Experiment.isin(["Exp27"])) & ~exp_report.label.isin(["accuracy","macro avg","weighted avg"])]
mbert_nurse.to_csv("Reports/languages/mbert_nurse.csv")
mbert_nurse

,index,label,precision,recall,f1-score,support,language,Experiment
80,0,0,0.874871,0.887723,0.881250,953,english,Exp27
81,1,1,0.524444,0.493724,0.508621,239,english,Exp27
85,5,0,0.811146,0.852033,0.831086,615,swahili,Exp27
86,6,1,0.491620,0.419048,0.452442,210,swahili,Exp27
90,10,0,0.861386,0.915789,0.887755,95,luo,Exp27
91,11,1,0.333333,0.222222,0.266667,18,luo,Exp27
95,15,0,0.895833,0.877551,0.886598,98,sheng,Exp27
96,16,1,0.478261,0.523810,0.500000,21,sheng,Exp27


In [28]:
#mBERT Nurse context languages (pretrained) - unlabeled
mbert_nurse_29 = exp_report[(exp_report.Experiment.isin(["Exp29"])) & ~exp_report.label.isin(["accuracy","macro avg","weighted avg"])]
mbert_nurse_29.to_csv("Reports/languages/mbert_nurse_29.csv")
mbert_nurse_29


,index,label,precision,recall,f1-score,support,language,Experiment
120,0,0,0.849356,0.899265,0.873598,953,english,Exp29
121,1,1,0.475410,0.364017,0.412322,239,english,Exp29
125,5,0,0.803982,0.853659,0.828076,615,swahili,Exp29
126,6,1,0.476744,0.390476,0.429319,210,swahili,Exp29
130,10,0,0.865385,0.947368,0.904523,95,luo,Exp29
131,11,1,0.444444,0.222222,0.296296,18,luo,Exp29
135,15,0,0.895238,0.959184,0.926108,98,sheng,Exp29
136,16,1,0.714286,0.476190,0.571429,21,sheng,Exp29


In [29]:
#mBERT System context languages (pretrained) - labeled
mbert_system = exp_report[(exp_report.Experiment.isin(["Exp26"])) & ~exp_report.label.isin(["accuracy","macro avg","weighted avg"])]
mbert_system.to_csv("Reports/languages/mbert_system.csv")
mbert_system

,index,label,precision,recall,f1-score,support,language,Experiment
60,0,0,0.846006,0.922350,0.882530,953,english,Exp26
61,1,1,0.516340,0.330544,0.403061,239,english,Exp26
65,5,0,0.793103,0.897561,0.842105,615,swahili,Exp26
66,6,1,0.511628,0.314286,0.389381,210,swahili,Exp26
70,10,0,0.855769,0.936842,0.894472,95,luo,Exp26
71,11,1,0.333333,0.166667,0.222222,18,luo,Exp26
75,15,0,0.882353,0.918367,0.900000,98,sheng,Exp26
76,16,1,0.529412,0.428571,0.473684,21,sheng,Exp26


In [30]:
#mBERT System context languages (pretrained) - unlabeled
mbert_system_28 = exp_report[(exp_report.Experiment.isin(["Exp28"])) & ~exp_report.label.isin(["accuracy","macro avg","weighted avg"])]
mbert_system_28.to_csv("Reports/languages/mbert_system_28.csv")
mbert_system_28

,index,label,precision,recall,f1-score,support,language,Experiment
100,0,0,0.853147,0.896118,0.874104,953,english,Exp28
101,1,1,0.481675,0.384937,0.427907,239,english,Exp28
105,5,0,0.804217,0.868293,0.835027,615,swahili,Exp28
106,6,1,0.496894,0.380952,0.431267,210,swahili,Exp28
110,10,0,0.882353,0.947368,0.913706,95,luo,Exp28
111,11,1,0.545455,0.333333,0.413793,18,luo,Exp28
115,15,0,0.906250,0.887755,0.896907,98,sheng,Exp28
116,16,1,0.521739,0.571429,0.545455,21,sheng,Exp28


In [31]:
#AfriBERT - Nurse context, no pretraining
afribert_nurse = exp_report[(exp_report.Experiment.isin(["Exp31"])) & ~exp_report.label.isin(["accuracy","macro avg","weighted avg"])]
afribert_nurse.to_csv("Reports/languages/afribert_nurse.csv")
afribert_nurse


,index,label,precision,recall,f1-score,support,language,Experiment
160,0,0,0.861616,0.895068,0.878024,953,english,Exp31
161,1,1,0.504950,0.426778,0.462585,239,english,Exp31
165,5,0,0.824615,0.871545,0.847431,615,swahili,Exp31
166,6,1,0.548571,0.457143,0.498701,210,swahili,Exp31
170,10,0,0.888889,0.926316,0.907216,95,luo,Exp31
171,11,1,0.500000,0.388889,0.437500,18,luo,Exp31
175,15,0,0.910891,0.938776,0.924623,98,sheng,Exp31
176,16,1,0.666667,0.571429,0.615385,21,sheng,Exp31


In [32]:
#AfriBERT - Nurse context, yes pretraining
afribert_nurse_35 = exp_report[(exp_report.Experiment.isin(["Exp35"])) & ~exp_report.label.isin(["accuracy","macro avg","weighted avg"])]
afribert_nurse_35.to_csv("Reports/languages/afribert_nurse_35.csv")
afribert_nurse_35

,index,label,precision,recall,f1-score,support,language,Experiment
200,0,0,0.849112,0.903463,0.875445,953,english,Exp35
201,1,1,0.483146,0.359833,0.412470,239,english,Exp35
205,5,0,0.822901,0.876423,0.848819,615,swahili,Exp35
206,6,1,0.552941,0.447619,0.494737,210,swahili,Exp35
210,10,0,0.875000,0.957895,0.914573,95,luo,Exp35
211,11,1,0.555556,0.277778,0.370370,18,luo,Exp35
215,15,0,0.885714,0.948980,0.916256,98,sheng,Exp35
216,16,1,0.642857,0.428571,0.514286,21,sheng,Exp35


In [33]:
#AfriBERT - system context, no pretraining
afribert_system = exp_report[(exp_report.Experiment.isin(["Exp30"])) & ~exp_report.label.isin(["accuracy","macro avg","weighted avg"])]
afribert_system.to_csv("Reports/languages/afribert_system.csv")
afribert_system


,index,label,precision,recall,f1-score,support,language,Experiment
140,0,0,0.851777,0.880378,0.865841,953,english,Exp30
141,1,1,0.449275,0.389121,0.417040,239,english,Exp30
145,5,0,0.812214,0.865041,0.837795,615,swahili,Exp30
146,6,1,0.511765,0.414286,0.457895,210,swahili,Exp30
150,10,0,0.887755,0.915789,0.901554,95,luo,Exp30
151,11,1,0.466667,0.388889,0.424242,18,luo,Exp30
155,15,0,0.896907,0.887755,0.892308,98,sheng,Exp30
156,16,1,0.500000,0.523810,0.511628,21,sheng,Exp30


In [34]:
#AfriBERT - system context, yes pretraining
afribert_system_34 = exp_report[(exp_report.Experiment.isin(["Exp34"])) & ~exp_report.label.isin(["accuracy","macro avg","weighted avg"])]
afribert_system_34.to_csv("Reports/languages/afribert_system_34.csv")
afribert_system_34

,index,label,precision,recall,f1-score,support,language,Experiment
180,0,0,0.855589,0.907660,0.880855,953,english,Exp34
181,1,1,0.513812,0.389121,0.442857,239,english,Exp34
185,5,0,0.817906,0.876423,0.846154,615,swahili,Exp34
186,6,1,0.542169,0.428571,0.478723,210,swahili,Exp34
190,10,0,0.885417,0.894737,0.890052,95,luo,Exp34
191,11,1,0.411765,0.388889,0.400000,18,luo,Exp34
195,15,0,0.928571,0.928571,0.928571,98,sheng,Exp34
196,16,1,0.666667,0.666667,0.666667,21,sheng,Exp34


In [35]:
#AfriBERT - no context
afribert_none = exp_report[(exp_report.Experiment.isin(["Exp38"])) & ~exp_report.label.isin(["accuracy","macro avg","weighted avg"])]
afribert_none.to_csv("Reports/languages/afribert_none.csv")
afribert_none

,index,label,precision,recall,f1-score,support,language,Experiment
220,0,0,0.844554,0.895068,0.869078,953,english,Exp38
221,1,1,0.450549,0.343096,0.389549,239,english,Exp38
225,5,0,0.799406,0.874797,0.835404,615,swahili,Exp38
226,6,1,0.493421,0.357143,0.414365,210,swahili,Exp38
230,10,0,0.870000,0.915789,0.892308,95,luo,Exp38
231,11,1,0.384615,0.277778,0.322581,18,luo,Exp38
235,15,0,0.898990,0.908163,0.903553,98,sheng,Exp38
236,16,1,0.550000,0.523810,0.536585,21,sheng,Exp38


In [36]:
exp_report.to_csv("Reports/languages/language_analysis_results.csv")